In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cp /content/drive/MyDrive/datasets/basel_preprocess.zip /content/

In [ ]:
#!pip install mysql-connector
!pip install Pillow --upgrade

In [ ]:
#the TensorFlow Model Garden repository
!git clone https://github.com/tensorflow/models.git


In [ ]:
!apt-get install protobuf-compiler python-lxml python-pil
!pip install Cython pandas tf-slim lvis

In [ ]:
#cd into 'TensorFlow/models/research'
%cd '/content/models/research'
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
import os
import sys
os.environ['PYTHONPATH']+=":/content/models"
sys.path.append("/content/models/research")

In [ ]:
!ls

In [ ]:
!python /content/models/research/object_detection/packages/tf2/setup.py build
!python /content/models/research/object_detection/packages/tf2/setup.py install

In [ ]:
#cd into 'TensorFlow/models/research/object_detection/builders/'
%cd '/content/models/research/object_detection/builders/'
!python model_builder_tf2_test.py
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
print('Done')

In [ ]:
%cd /content/

In [ ]:
!wget "https://raw.githubusercontent.com/abundis-rmn2/graffiti_detection_OD_TF/main/labelmap.pbtxt"
!wget "https://data.abundis.com.mx/inference_graph.tar.gz"
!wget "https://data.abundis.com.mx/config.json"
!tar xvzf inference_graph.tar.gz
!ls

In [ ]:
import time
import argparse
#import mysql.connector
import json

import requests
import io
import os
import numpy as np
import glob
import json

from six import BytesIO
from PIL import Image
Image.LOAD_TRUNCATED_IMAGES = True

import tensorflow as tf
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
import ftplib

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.
  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.
  Args:
    path: a file path (this can be local or on colossus)
  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis, ...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key: value[0, :num_detections].numpy()
                 for key, value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
      output_dict['detection_masks'], output_dict['detection_boxes'],
      image.shape[0], image.shape[1])
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

  return output_dict

def directory_exists(dir,ftp):
    filelist = []
    ftp.retrlines('LIST',filelist.append)
    for f in filelist:
        if f.split()[-1] == dir and f.upper().startswith('D'):
            return True
    return False

'''def DataUpload(local_dir, target_dir):
    ftp_server.cwd('/media/exported_images')
    if directory_exists(target_dir, ftp_server) is False:  # (or negate, whatever you prefer for readability)
        print(target_dir)
        ftp_server.mkd(target_dir)
    ftp_server.cwd(target_dir)
    # https://stackoverflow.com/questions/67520579/uploading-a-files-in-a-folder-to-ftp-using-python-ftplib
    print("Uploading exported batch")
    toFTP = os.listdir(local_dir)
    for filename in toFTP:
        if filename not in ftp_server.nlst():
            print("Uploading: ")
            with open(os.path.join(local_dir, filename), 'rb') as file:  # Here I open the file using it's  full path
                ftp_server.storbinary(f'STOR {filename}', file)  # Here I store the file in the FTP using only it's name as I intended
            print(filename)
        else:
            print("File already exist")
    ftp_server.quit()'''

In [ ]:
labelmap_path = 'labelmap.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(labelmap_path, use_display_name=True)
print("Load labelmap")
tf.keras.backend.clear_session()
model = tf.saved_model.load('inference_graph/saved_model')
print("Load inference graph")

global_time = time.time()



#c = open("config.json")
#config = json.load(c)


Load labelmap
Load inference graph


In [ ]:
os.makedirs("./downloaded_images/basel_test", 0o777)
!mv /content/basel_preprocess.zip /content/downloaded_images/
#!rm -r /content/downloaded_images/basel_preprocess
!unzip "/content/downloaded_images/basel_preprocess.zip" -d "/content/downloaded_images/"

In [ ]:
MUID = "basel_preprocess"
json_name = 'normalized_dataset_info.json'
output_name = 'output_json.json'

# Load Dataset Info
if (os.path.exists(f'./downloaded_images/{MUID}/{json_name}')):
    with open(f'./downloaded_images/{MUID}/{json_name}', "r") as file:
        pano_json = json.load(file)
else:
    print("JSON for the dataset is missing!")

# Load Output Info
if (os.path.exists(f'./downloaded_images/{MUID}/{output_name}')):
    with open(f'./downloaded_images/{MUID}/{output_name}', "r") as file:
        output_json = json.load(file)
        #last_item = output_json[-1]
        #print(last_item)
        #last_id = last_item['id']
        pano_json_new = output_json
else:
    print("There is no JSON with Output History!")
    #last_id = 0
    pano_json_new = []

dir_exist = os.path.exists("./exported_images/" + MUID)
if not dir_exist:
    #os.makedirs(user_dir, 0o777)
    os.makedirs("./exported_images/" + MUID, 0o777)
    print("The dir was created")
else:
    print("The dir already exist")

dir_exist = os.path.exists("./downloaded_images/")
if not dir_exist:
    #os.makedirs(user_dir, 0o777)
    os.makedirs("./downloaded_images/", 0o777)
    print("The dir was created")
else:
    print("The dir already exist")


    print("Looking for images in MUID:", MUID)
    images = glob.glob(f'./downloaded_images/{MUID}/*.jpg')
    print("MUID found: ", len(images))


# Get the current time
start_time = time.time()
# Set the interval in seconds
interval = 120

for img in images:
  if(images.index(img) > 5960):
    os.system('clx')
    print(f'{images.index(img)}/{len(images)}')
    try:
        image_np = load_image_into_numpy_array(img)
        output_dict = run_inference_for_single_image(model, image_np)
        '''vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            output_dict['detection_boxes'],
            output_dict['detection_classes'],
            output_dict['detection_scores'],
            category_index,
            instance_masks=output_dict.get('detection_masks_reframed', None),
            use_normalized_coordinates=True,
            line_thickness=8)
        im = Image.fromarray(image_np)
        im.save(f'./exported_images/{MUID}/{id(img)}.jpg')'''
    except:
        pass

    pano = ""
    c = 1
    for d_class, d_score, d_boxes in zip(output_dict['detection_classes'][:5], output_dict['detection_scores'][:5],output_dict['detection_boxes'][:5]):
      if d_score > 0.5:
        d_class_name = category_index[d_class]['name']
        #print('{0} with score {1}'.format(d_class_name, d_score))

        #Structure Output for JSON
        obj = {d_class_name: {
              'score': float(d_score),
              'side': img[-5:-4], #f'./downloaded_images/{MUID}/*.jpg'
              'pos': d_boxes.tolist()
              }}

        #Load JSON Data (if img ids matching > keep)

        #PREVENT DUPLICATES!!!
        
        for pano in pano_json:
          if pano['id'] == img.replace(f'./downloaded_images/{MUID}/','')[:-6]:
            if pano not in pano_json_new:
            # Add the list named "objects"
              pano['objects'].append(obj)
            else:
              pano_json_new[pano]['objects'].append(obj)
            break
    if not pano == "":
      pano_json_new.append(pano)

    elapsed_time = time.time() - start_time
    print(f'Elapsed time: {elapsed_time}')
    if elapsed_time >= interval:
      # Execute your code here
      inference_json = json.dumps(pano_json_new, indent=4)
      with open(f'/content/drive/MyDrive/datasets/{output_name}', 'w') as outfile:
        current_time = time.time()
        outfile.write(inference_json)
      print("----------- Saving... -----------")
      # Reset the start time
      start_time = time.time()
    # Wait for a short time to avoid consuming too much CPU time
    time.sleep(0.1)
      
inference_json = json.dumps(pano_json_new, indent=4)
with open(f'/content/drive/MyDrive/datasets/{output_name}', 'w') as outfile:
    outfile.write(inference_json)

The dir already exist
The dir already exist
Looking for images in MUID: basel_preprocess
MUID found:  10272
5961/10272
Elapsed time: 1.3283898830413818
5962/10272
Elapsed time: 2.007566213607788
5963/10272
Elapsed time: 2.6909778118133545
5964/10272
Elapsed time: 3.367452621459961
5965/10272
Elapsed time: 4.026472091674805
5966/10272
Elapsed time: 4.607198476791382
5967/10272
Elapsed time: 5.568461656570435
5968/10272
Elapsed time: 6.487635374069214
5969/10272
Elapsed time: 7.162556409835815
5970/10272
Elapsed time: 7.834930419921875
5971/10272
Elapsed time: 8.507859230041504
5972/10272
Elapsed time: 9.18203067779541
5973/10272
Elapsed time: 10.11753511428833
5974/10272
Elapsed time: 11.039308071136475
5975/10272
Elapsed time: 12.308043241500854
5976/10272
Elapsed time: 13.55767011642456
5977/10272
Elapsed time: 14.793631076812744
5978/10272
Elapsed time: 15.960301876068115
5979/10272
Elapsed time: 17.133535623550415
5980/10272
Elapsed time: 18.84387469291687
5981/10272
Elapsed time: 2

KeyboardInterrupt: ignored